# Poisoning Attacks against Machine Learning models

In this tutorial we will experiment with **adversarial poisoning attacks** against a Support Vector Machine (SVM) with Radial Basis Function (RBF) kernel.

Poisoning attacks are performed at *train time* by injecting *carefully crafted samples* that alter the classifier decision function so that its accuracy decreases.

As in the previous tutorials, we will first create and train the classifier, evaluating its performance in the standard scenario, *i.e. not under attack*.
The poisoning attack will also need a *validation set* to verify the classifier performance during the attack, so we split the training set furtherly in two.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zangobot/teaching_material/blob/HEAD/Extra-Poisoning.ipynb)

In [ ]:
%%capture --no-stderr --no-display
# NBVAL_IGNORE_OUTPUT

try:
    import secml
except ImportError:
    %pip install git+https://github.com/pralab/secml
    %pip install foolbox

In [ ]:
random_state = 999

n_features = 2  # Number of features
n_samples = 300  # Number of samples
centers = [[-1, -1], [+1, +1]]  # Centers of the clusters
cluster_std = 0.9  # Standard deviation of the clusters

from secml.data.loader import CDLRandomBlobs

dataset = CDLRandomBlobs(n_features=n_features,
                         centers=centers,
                         cluster_std=cluster_std,
                         n_samples=n_samples,
                         random_state=random_state).load()

n_tr = 100  # Number of training set samples
n_val = 100  # Number of validation set samples
n_ts = 100  # Number of test set samples

# Split in training, validation and test
from secml.data.splitter import CTrainTestSplit

splitter = CTrainTestSplit(
    train_size=n_tr + n_val, test_size=n_ts, random_state=random_state)
tr_val, ts = splitter.split(dataset)
splitter = CTrainTestSplit(
    train_size=n_tr, test_size=n_val, random_state=random_state)
tr, val = splitter.split(dataset)

# Normalize the data
from secml.ml.features import CNormalizerMinMax

nmz = CNormalizerMinMax()
tr.X = nmz.fit_transform(tr.X)
val.X = nmz.transform(val.X)
ts.X = nmz.transform(ts.X)

# Metric to use for training and performance evaluation
from secml.ml.peval.metrics import CMetricAccuracy

metric = CMetricAccuracy()

# Creation of the multiclass classifier
from secml.ml.classifiers import CClassifierSVM
from secml.ml.kernels import CKernelRBF

clean_clf = CClassifierSVM(kernel=CKernelRBF(gamma=10), C=1)

# We can now fit the classifier
clean_clf.fit(tr.X, tr.Y)
print("Training of classifier complete!")

# Compute predictions on a test set
y_pred = clean_clf.predict(ts.X)

## Generation of Poisoning Samples

We are going to generate an adversarial example against the SVM classifier using the **gradient-based** algorithm for generating poisoning attacks proposed in:
 
  > [[biggio12-icml]](https://arxiv.org/abs/1206.6389)
  > Biggio, B., Nelson, B. and Laskov, P., 2012. Poisoning attacks against 
  > support vector machines. In ICML 2012.

  > [[biggio15-icml]](https://arxiv.org/abs/1804.07933)
  > Xiao, H., Biggio, B., Brown, G., Fumera, G., Eckert, C. and Roli, F., 2015. 
  > Is feature selection secure against training data poisoning?. In ICML 2015.

  > [[demontis19-usenix]](
  > https://www.usenix.org/conference/usenixsecurity19/presentation/demontis)
  > Demontis, A., Melis, M., Pintor, M., Jagielski, M., Biggio, B., Oprea, A., 
  > Nita-Rotaru, C. and Roli, F., 2019. Why Do Adversarial Attacks Transfer? 
  > Explaining Transferability of Evasion and Poisoning Attacks. In 28th Usenix 
  > Security Symposium, Santa Clara, California, USA.

To compute a poisoning point, a bi-level optimization problem has to be solved, namely:

$$
\begin{aligned}
\max_{x_c}& A(D_{val}, \mathbf{w}^\ast) = \sum_{j=1}^m \ell(y_j, \mathbf{x_\mathit{j}}, \mathbf{w}^\ast)\\
&s.t. \mathbf{w}^\ast \in \underset{\mathbf{w}}{\operatorname{arg min}} \textit{L} (D_{tr} \cup (\mathbf{x}_c, y_c), \mathbf{w})
\end{aligned}
$$

Where $\mathbf{x_c}$ is the poisoning point, $A$ is the attacker objective function, $L$ is the classifier training function.
Moreover, $D_{tr}$ is the training dataset and $D_{val}$ is the validation dataset.
The former problem, along with the poisoning point $\mathbf{x}_c$ is used to train the classifier on the poisoned data, while the latter is used to evaluate the performance on the untainted data.

The former equation depends on the classifier weights, which in turns, depends on the poisoning point.

This attack is implemented in SecML by different subclasses of the `CAttackPoisoning`.
For the purpose of attacking a SVM classifier we use the `CAttackPoisoningSVM` class.

As done for the [evasion attacks](03-Evasion.ipynb), let's specify the parameters first. We set the bounds of the attack space to the known feature space given by validation dataset.
Lastly, we chose the solver parameters for this specific optimization problem.

Let's start visualizing the objective function considering a single poisoning point.

In [ ]:
lb, ub = val.X.min(), val.X.max()  # Bounds of the attack space. Can be set to `None` for unbounded

# Should be chosen depending on the optimization problem
solver_params = {
    'eta': 0.05,
    'eta_min': 0.05,
    'eta_max': None,
    'max_iter': 100,
    'eps': 1e-6
}

from secml.adv.attacks import CAttackPoisoningSVM

pois_attack = CAttackPoisoningSVM(classifier=clean_clf,
                                  training_data=tr,
                                  val=val,
                                  lb=lb, ub=ub,
                                  solver_params=solver_params,
                                  random_seed=random_state)

# chose and set the initial poisoning sample features and label
xc = tr[0, :].X
yc = tr[0, :].Y
pois_attack.x0 = xc
pois_attack.xc = xc
pois_attack.yc = yc

print("Initial poisoning sample features: {:}".format(xc.ravel()))
print("Initial poisoning sample label: {:}".format(yc.item()))

from secml.figure import CFigure
# Only required for visualization in notebooks
%matplotlib inline

fig = CFigure(4, 5)

grid_limits = [(lb - 0.1, ub + 0.1),
               (lb - 0.1, ub + 0.1)]

fig.sp.plot_ds(tr)

# highlight the initial poisoning sample showing it as a star
fig.sp.plot_ds(tr[0, :], markers='*', markersize=16)

fig.sp.title('Attacker objective and gradients')
fig.sp.plot_fun(
    func=pois_attack.objective_function,
    grid_limits=grid_limits, plot_levels=False,
    n_grid_points=10, colorbar=True)

# plot the box constraint
from secml.optim.constraints import CConstraintBox

box = fbox = CConstraintBox(lb=lb, ub=ub)
fig.sp.plot_constraint(box, grid_limits=grid_limits,
                       n_grid_points=10)

fig.tight_layout()
fig.show()

Now, we set the desired number of adversarial points to generate, 20 in this example.

In [ ]:
steps = 20
n_poisoning_points = steps  # Number of poisoning points to generate
pois_attack.n_points = n_poisoning_points

# Run the poisoning attack
print("Attack started...")
pois_y_pred, pois_scores, pois_ds, f_opt = pois_attack.run(ts.X, ts.Y)
print("Attack complete!")

# Evaluate the accuracy of the original classifier
acc = metric.performance_score(y_true=ts.Y, y_pred=y_pred)
# Evaluate the accuracy after the poisoning attack
pois_acc = metric.performance_score(y_true=ts.Y, y_pred=pois_y_pred)

print("Original accuracy on test set: {:.2%}".format(acc))
print("Accuracy after attack on test set: {:.2%}".format(pois_acc))

We can see that the classifiers has been successfully attacked.
To increase the attack power, more poisoning points can be crafted, at the expense of a much slower optimization process.

Let's now visualize the attack on a 2D plane.
We need to train a copy of the original classifier on the join between the training set and the poisoning points.

In [ ]:
# Training of the poisoned classifier
poisoned_clf = clean_clf.deepcopy()
pois_tr = tr.append(pois_ds)  # Join the training set with the poisoning points
poisoned_clf.fit(pois_tr.X, pois_tr.Y)

# Define common bounds for the subplots
min_limit = min(pois_tr.X.min(), ts.X.min())
max_limit = max(pois_tr.X.max(), ts.X.max())
grid_limits = [[min_limit, max_limit], [min_limit, max_limit]]

fig = CFigure(10, 10)

fig.subplot(2, 2, 1)
fig.sp.title("Original classifier (training set)")
fig.sp.plot_decision_regions(
    clean_clf, n_grid_points=200, grid_limits=grid_limits)
fig.sp.plot_ds(tr, markersize=5)
fig.sp.grid(grid_on=False)

fig.subplot(2, 2, 2)
fig.sp.title("Poisoned classifier (training set + poisoning points)")
fig.sp.plot_decision_regions(
    poisoned_clf, n_grid_points=200, grid_limits=grid_limits)
fig.sp.plot_ds(tr, markersize=5)
fig.sp.plot_ds(pois_ds, markers=['*', '*'], markersize=12)
fig.sp.grid(grid_on=False)

fig.subplot(2, 2, 3)
fig.sp.title("Original classifier (test set)")
fig.sp.plot_decision_regions(
    clean_clf, n_grid_points=200, grid_limits=grid_limits)
fig.sp.plot_ds(ts, markersize=5)
fig.sp.text(0.05, -0.25, "Accuracy on test set: {:.2%}".format(acc),
            bbox=dict(facecolor='white'))
fig.sp.grid(grid_on=False)

fig.subplot(2, 2, 4)
fig.sp.title("Poisoned classifier (test set)")
fig.sp.plot_decision_regions(
    poisoned_clf, n_grid_points=200, grid_limits=grid_limits)
fig.sp.plot_ds(ts, markersize=5)
fig.sp.text(0.05, -0.25, "Accuracy on test set: {:.2%}".format(pois_acc),
            bbox=dict(facecolor='white'))
fig.sp.grid(grid_on=False)

fig.show()

We can see how the SVM classifier decision functions *changes* after injecting the adversarial poisoning points (blue and red stars).
 
For more details about poisoning adversarial attacks please refer to:

  > [[biggio18-pr]](https://arxiv.org/abs/1712.03141)
  > Biggio, B. and Roli, F., 2018. Wild patterns: Ten years after the rise of 
  > adversarial machine learning. In Pattern Recognition.

# Performance of classifier for more poisoning samples

Instead of hard-coding the number of points, we can compute poisoning for different fractions of points, as we did for evasion attacks.
After each poisoning, we train a classifier with both clean and poisoned samples and we compute the accuracy.

In [ ]:
from secml.array import CArray

n_poisonous_samples = CArray([1, 10, steps, 75, 100])


poisoned_clfs = []
test_accuracies = CArray.zeros(n_poisonous_samples.shape[0])

for i, p in enumerate(n_poisonous_samples):
    print(f'Computing poisoning for {p} points')
    pois_attack = ... #Instatiate the poisoning attack as seen before
    pois_attack.n_points = p
    _, _, ds, _ = pois_attack.run(ts.X, ts.Y)
    poisoned_clf = clean_clf.deepcopy()
    pois_tr = tr.append(ds)
    poisoned_clf.fit(pois_tr.X, pois_tr.Y)
    y_pred = ... # compute predictions on the test set using the poisoned model
    accuracy = ... # Compute the accuracy on the test set
    test_accuracies[i] = accuracy
    poisoned_clfs.append(poisoned_clf)
    print(f'Test accuracy is {accuracy}')


In [ ]:
%matplotlib inline
print(test_accuracies)
fig = CFigure()
fig.sp.plot(n_poisonous_samples, test_accuracies)
fig.sp.xlabel('Poison samples')
fig.sp.xticks(n_poisonous_samples)
fig.sp.ylabel('Test accuracy')
fig.sp.ylim([0,1])
fig.show()

# Robustness of poisoned models

Test accuracy is reduced the more samples we inject inside the training set of the classifier.
What happens if we attack these poisoned models?
What do you expect it will happen?

In [ ]:
from secml.adv.seceval import CSecEval
from secml.adv.attacks import CFoolboxPGDL2

sec_evals = []
epsilons = CArray([0.05, 0.1, 0.2, 0.5])
fig = CFigure()

abs_stepsize = 0.05

for p_clf, n in zip(poisoned_clfs, n_poisonous_samples):
    print(f'Security evaluation of classifier with {n} poisoned samples...')
    attack = ...#  Instantiate a CFoolboxPGDL2 attack
    sec_eval = ... # instantiate CSecEval object
    # Run the security evaluation as done in the previous laboratory
    fig.sp.plot_sec_eval(sec_eval.sec_eval_data, label=f'{n} p')
    sec_evals.append(sec_eval)
fig.sp.ylim([0,1])
fig.show()